In [3]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

## Group Assignment
### Team Number: 17
### Team Member Names: John, Gen, Tim
### Team Strategy Chosen: Risky 

Requirements for this assignment:
- Code needs to be dynamic
- Must read in a csv file containing a finite number of stock tickers (with an example csv file in this repo)
- Portfolio must contain only US listed stocks
- Must have an average daily volume of at least 10 000 shares as calculated based on the time interval July 2, 2021 to October 22, 2021
- Pick a minimum 10 maximum 20 stocks for portfolio.
- If we choose n stocks, each stock must make up minimum (100/(2n))% of the portfolio when weighted by value as of closing prices on November 26, 2021
- No individual stock may make up more than 35% of the portfolio when weighted by value
- We have $100000 USD to spend on portfolio and all must be spent
- When code is run with the .csv file, it must create a DataFrame "FinalPortfolio"
    - With this portfolio, index starts at 1 and ends at the number of stocks that our code chooses.
    - Headings must be: Ticker, Price, Shares, Value, Weight
        - Ticker is the ticker selected
        - Price is the price on November 26, 2021
        - Shares is the number of shares purchased (can be fractional)
        - Value is the total value of those shares
        - Weighted is the weight that the value of shares represents relative to the value of the portfolio (which is $100,000)
    - Needs to show that the total adds up to $100,000
    - Also need to show the weights add to 100%
    - This DataFrame must be printed to the screen as the second to last output to the screen.
- After the DataFrame, one final DataFrame called "Stocks" which has the same index as "Final Portfolio" must be mad
    - Only has the Tickers and Shares from "Final Portfolio"
    - Must output this DataFrame to a csv file titled "Stocks_Group_XX.csv"

In [4]:
stocks_from_csv = pd.read_csv('Tickers.csv')
stocks_from_csv


,AAPL
0,ABBV
1,ABT
2,ACN
3,AGN
4,AIG
...,...
56,TXN
57,UNH
58,UNP
59,UPS


In [5]:
stocks_lst = stocks_from_csv.iloc[:,0]
stocks_lst

0     ABBV
1      ABT
2      ACN
3      AGN
4      AIG
      ... 
56     TXN
57     UNH
58     UNP
59     UPS
60     USB
Name: AAPL, Length: 61, dtype: object

In [6]:
stocks_from_csv.columns[0]

'AAPL'

In [7]:
current_stock=yf.Ticker(stocks_lst[0])
current_stock.info['market']

'us_market'

In [8]:
us_stock_lst = []
i=0
while i < stocks_from_csv.index.size:
    if (yf.Ticker(stocks_lst[i]).info['regularMarketPrice'] != None) and (yf.Ticker(stocks_lst[i]).info['market']=='us_market'):
        us_stock_lst.append(yf.Ticker(stocks_lst[i]))
    i+=1

In [9]:
us_stock_lst

[yfinance.Ticker object <ABBV>,
 yfinance.Ticker object <ABT>,
 yfinance.Ticker object <ACN>,
 yfinance.Ticker object <AIG>,
 yfinance.Ticker object <AMZN>,
 yfinance.Ticker object <AXP>,
 yfinance.Ticker object <BA>,
 yfinance.Ticker object <BAC>,
 yfinance.Ticker object <BIIB>,
 yfinance.Ticker object <BK>,
 yfinance.Ticker object <BLK>,
 yfinance.Ticker object <BMY>,
 yfinance.Ticker object <C>,
 yfinance.Ticker object <CAT>,
 yfinance.Ticker object <CL>,
 yfinance.Ticker object <CMCSA>,
 yfinance.Ticker object <COF>,
 yfinance.Ticker object <COP>,
 yfinance.Ticker object <COST>,
 yfinance.Ticker object <CSCO>,
 yfinance.Ticker object <CVS>,
 yfinance.Ticker object <GM>,
 yfinance.Ticker object <GOOG>,
 yfinance.Ticker object <JPM>,
 yfinance.Ticker object <KMI>,
 yfinance.Ticker object <KO>,
 yfinance.Ticker object <LLY>,
 yfinance.Ticker object <LMT>,
 yfinance.Ticker object <MO>,
 yfinance.Ticker object <MON>,
 yfinance.Ticker object <MRK>,
 yfinance.Ticker object <MS>,
 yfinance

In [83]:
valid_stock_list = []

# time interval of July 02, 2021 to October 22, 2021
start_date = "2021-07-02"
end_date = "2021-10-22"

# Finds the average daily volume of a stock
def avg_daily_volume(ticker):
    ticker_history = ticker.history(start=start_date,end=end_date,interval='1d')
    ticker_vol_avg = ticker_history['Volume'].mean()
    return ticker_vol_avg
    

# iterates through us_stock_lst and checks if the stock has an average daily volume of at least 10,000 shares and appends to a list
def volume_filter(ticker_lst):
    for i in us_stock_lst:
        if avg_daily_volume(i) >= 10_000:
            valid_stock_list.append(i)
        else:
            continue
    print(valid_stock_list)

volume_filter(us_stock_lst)







[yfinance.Ticker object <ABBV>, yfinance.Ticker object <ABT>, yfinance.Ticker object <ACN>, yfinance.Ticker object <AIG>, yfinance.Ticker object <AMZN>, yfinance.Ticker object <AXP>, yfinance.Ticker object <BA>, yfinance.Ticker object <BAC>, yfinance.Ticker object <BIIB>, yfinance.Ticker object <BK>, yfinance.Ticker object <BLK>, yfinance.Ticker object <BMY>, yfinance.Ticker object <C>, yfinance.Ticker object <CAT>, yfinance.Ticker object <CL>, yfinance.Ticker object <CMCSA>, yfinance.Ticker object <COF>, yfinance.Ticker object <COP>, yfinance.Ticker object <COST>, yfinance.Ticker object <CSCO>, yfinance.Ticker object <CVS>, yfinance.Ticker object <GM>, yfinance.Ticker object <GOOG>, yfinance.Ticker object <JPM>, yfinance.Ticker object <KMI>, yfinance.Ticker object <KO>, yfinance.Ticker object <LLY>, yfinance.Ticker object <LMT>, yfinance.Ticker object <MO>, yfinance.Ticker object <MON>, yfinance.Ticker object <MRK>, yfinance.Ticker object <MS>, yfinance.Ticker object <MSFT>, yfinance.

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here. 